<a href="https://colab.research.google.com/github/Mariana-zy/SyS-2025-2S/blob/main/Taller2/Punto_2_3_y_2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Punto 2.3:**

Demuestre si los siguientes sistemas de la forma $y = \mathcal{H}\{x\}$, son sistemas lineales e invariantes en el tiempo (SLIT).

Para que un sistema $y = \mathcal{H}\{x\}$ sea SLIT, debe cumplir dos propiedades:

1.  **Linealidad:** Debe satisfacer el principio de superposición. Si $y_1 = \mathcal{H}\{x_1\}$ y $y_2 = \mathcal{H}\{x_2\}$, entonces $\mathcal{H}\{ax_1 + bx_2\} = ay_1 + by_2$ para cualquier constante $a, b$.

2.  **Invariancia en el Tiempo:** Si la entrada se desplaza en el tiempo, la salida se desplaza en la misma cantidad. Si $y[n] = \mathcal{H}\{x[n]\}$, entonces $\mathcal{H}\{x[n - n_0]\} = y[n - n_0]$.

* $y[n] = x[n]/3 + 2x[n-1] - y[n-1].$



In [ ]:
import numpy as np
from scipy.signal import lfilter, medfilt
import matplotlib.pyplot as plt

# Definir los coeficientes
b = [1/3, 2]  # Coeficientes de x[n], x[n-1]
a = [1, 1]    # Coeficientes de y[n], y[n-1]

# Crear señales de prueba
n = np.arange(0, 20)
x1 = np.cos(n * 0.2)
x2 = np.sin(n * 0.5)
const_a, const_b = 2.0, -3.0

# 1. Prueba de Linealidad

# H{a*x1 + b*x2}
x_super = const_a * x1 + const_b * x2
y_super = lfilter(b, a, x_super)

# a*H{x1} + b*H{x2}
y1 = lfilter(b, a, x1)
y2 = lfilter(b, a, x2)
y_linear_combo = const_a * y1 + const_b * y2

is_linear = np.allclose(y_super, y_linear_combo)
print(f"¿Es Lineal?: {is_linear}")

# 2. Prueba de Invariancia en el Tiempo

n0 = 5  # Desplazamiento

# H{x[n-n0]} (Entrada desplazada)
x_shifted = np.roll(x1, n0)
x_shifted[:n0] = 0  # Forzar ceros al inicio
y_from_shifted_input = lfilter(b, a, x_shifted)

# y[n-n0] (Salida desplazada causalmente)
y_shifted_output = np.roll(y1, n0)
y_shifted_output[:n0] = 0 # Forzar ceros al inicio

is_ti = np.allclose(y_from_shifted_input, y_shifted_output)
print(f"¿Es Invariante en el Tiempo?: {is_ti}")
print(f"Conclusión: {'SÍ es SLIT' if is_linear and is_ti else 'NO es LTI'}")

¿Es Lineal?: True
¿Es Invariante en el Tiempo?: True
Conclusión: SÍ es SLIT


* $y[n] = \sum_{k=-\infty}^{n} x^2[k].$

In [ ]:
def system2(x):
    # np.cumsum es el acumulador (suma de k=0 a n)
    # Asumimos que la señal es 0 para k < 0
    return np.cumsum(x**2)

# Crear señales de prueba
n = np.arange(0, 20)
x1 = np.cos(n * 0.2)
a_const = 2.0 # Constante para prueba linealidad

# 1. Prueba de Linealidad

# H{a*x}
x_scaled = a_const * x1
y_scaled = system2(x_scaled)

# a*H{x}
y1 = system2(x1)
a_y1 = a_const * y1

is_linear = np.allclose(y_scaled, a_y1)
print(f"¿Es Lineal?: {is_linear}")

# 2. Prueba de Invariancia en el Tiempo

n0 = 5  # Desplazamiento

# H{x[n-n0]}
x_shifted = np.roll(x1, n0)
x_shifted[:n0] = 0 # Forzar ceros al inicio
y_from_shifted_input = system2(x_shifted)

# y[n-n0]
y_shifted_output = np.roll(y1, n0)
y_shifted_output[:n0] = 0 # Forzar ceros al inicio

is_ti = np.allclose(y_from_shifted_input, y_shifted_output)
print(f"¿Es Invariante en el Tiempo?: {is_ti}")
print(f"Conclusión: {'SÍ es SLIT' if is_linear and is_ti else 'NO es SLIT'}")

¿Es Lineal?: False
¿Es Invariante en el Tiempo?: True
Conclusión: NO es SLIT


* $y[n] = \text{median}(x[n])$; donde median es la función mediana sobre una ventana de tamaño 3.

In [ ]:
def system3(x):
    # Usamos una ventana causal de tamaño 3 [n, n-1, n-2]
    # Lo simulamos manualmente para controlar la causalidad
    y = np.zeros_like(x)
    # Rellenamos el inicio para manejar los bordes
    x_padded = np.concatenate(([0, 0], x))
    for i in range(len(x)):
        # Ventana: x_padded[i], x_padded[i+1], x_padded[i+2]
        # que corresponde a x[i-2], x[i-1], x[i]
        window = [x_padded[i], x_padded[i+1], x_padded[i+2]]
        y[i] = np.median(window)
    return y

# 1. Prueba de Linealidad

# Usamos un contraejemplo
x1 = np.array([0., 1., 10., 0., 0., 0.])
x2 = np.array([0., 10., 1., 0., 0., 0.])

# H{x1} + H{x2}
y1 = system3(x1)
y2 = system3(x2)
y_sum = y1 + y2

# H{x1 + x2}
x_sum = x1 + x2
y_from_sum = system3(x_sum)

is_linear = np.allclose(y_sum, y_from_sum)
print(f"¿Es Lineal?: {is_linear}")

# 2. Prueba de Invariancia en el Tiempo

n0 = 2 # Desplazamiento

# H{x[n-n0]}
x_shifted = np.roll(x1, n0)
x_shifted[:n0] = 0 # Forzar ceros al inicio
y_from_shifted_input = system3(x_shifted)

# y[n-n0]
y_shifted_output = np.roll(y1, n0)
y_shifted_output[:n0] = 0 # Forzar ceros al inicio

is_ti = np.allclose(y_from_shifted_input, y_shifted_output)
print(f"¿Es Invariante en el Tiempo?: {is_ti}")
print(f"Conclusión: {'SÍ es SLIT' if is_linear and is_ti else 'NO es SLIT'}")

¿Es Lineal?: False
¿Es Invariante en el Tiempo?: True
Conclusión: NO es SLIT



* $y(t) = Ax(t) + B; A, B \in \mathbb{R}.$

In [ ]:
def system4(x, A, B):
    return A * x + B

# Parámetros del sistema
A = 2.0
B = 5.0 # B diferente de 0 para demostrar la no-linealidad

# Crear señales de prueba
t = np.linspace(0, 1, 100)
x1 = np.cos(t * 2 * np.pi)
x2 = np.sin(t * 4 * np.pi)

# 1. Prueba de Linealidad

# H{x1 + x2}
x_sum = x1 + x2
y_from_sum = system4(x_sum, A, B) # A*(x1+x2) + B

# H{x1} + H{x2}
y1 = system4(x1, A, B) # A*x1 + B
y2 = system4(x2, A, B) # A*x2 + B
y_sum = y1 + y2        # A*(x1+x2) + 2*B

is_linear = np.allclose(y_from_sum, y_sum)
print(f"¿Es Lineal?: {is_linear}")

# 2. Prueba de Invariancia en el Tiempo

n0 = 10 # Desplazamiento (en muestras)

# H{x(t-t0)}
x_shifted = np.roll(x1, n0)
x_shifted[:n0] = 0 # <-- CORRECCIÓN: Desplazamiento causal
y_from_shifted_input = system4(x_shifted, A, B)

# y(t-t0)
y1 = system4(x1, A, B) # Recalculamos y1 por si acaso
y_shifted_output = np.roll(y1, n0)
y_shifted_output[:n0] = B # <-- CORRECCIÓN: Un shift causal de y=Ax+B
                          # implica que y es B cuando x es 0.

is_ti = np.allclose(y_from_shifted_input, y_shifted_output)
print(f"¿Es Invariante en el Tiempo?: {is_ti}")
print(f"Conclusión: {'SÍ es SLIT' if is_linear and is_ti else 'NO es SLIT'}")

¿Es Lineal?: False
¿Es Invariante en el Tiempo?: True
Conclusión: NO es SLIT


# **Punto 2.4:**

**Ejercicio 1:** Hallar la salida $ y[n] $ de un SLIT ante la entrada $ x[n]=\{-15, 5, -3^\dagger, 0, 5, 7, -1\} $, con respuesta al impulso $ h[n]=\{1, -2, 0^\dagger, 1, -2\} $, donde $ n \in \{0, \pm 1, \pm 2, \ldots, \pm N\} $ y $ n=0 $ para $ x[n]^\dagger $. Nota: Utilizar método gráfico para encontrar la salida y comprobar con simulación en Python.



In [ ]:
%matplotlib inline
from ipywidgets import interact #crear graficos interactivos
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal #crear señales prototipo en tiempo discreto

#crear señales
#h = np.array([1,-2,0,1,-2]).reshape(-1,1)#np.r_[np.zeros((5,1)),np.ones((5,1)),np.zeros((5,1))] # se genera un pulso rectangular
hu = np.array([-1, 6, -10, 3, 1, -10, 2, 5])
h = np.diff(hu).reshape(-1,1)
x = np.array([-15, 5, -3, 0, 5, 7, -1]).reshape(-1,1)#signal.triang(12).reshape(-1,1) # se genera una señal triangular y se trabaja con arreglos tipo columna
#funcion convolucion
def convgraf(h,x): #función general para pintar proceso de convolución de forma interactiva
   lx = len(x)
   lh = len(h)
   M = 2*lx+lh # se genera un vector de tamaño completo para visualizar todo el proceso paso a paso
   xm = np.zeros((M,M))
   hm = np.r_[np.zeros((lx,1)),h,np.zeros((lx,1))]
   ym = np.zeros((M,M))
   for i in range(M-lx+1):
     #print(i)
     xm[i:i+lx,i] = np.flip(x).reshape(-1)#filp realizar el proceso de reflejo sobre arreglos discretos
     ym[i,i] = xm[:,i].T.dot(hm) #multiplicacion entre vectores -> producto punto
   return xm, ym, hm

def plot_conv(k,xm,ym,hm): #dibujar arreglos
     plt.stem(xm[:,k],markerfmt='+',label='$x[k]$')
     plt.stem(hm,linefmt='g',markerfmt='.',label='$h[n-k]$')
     plt.stem(ym[:k,:].sum(axis=0),markerfmt='s',linefmt='r',label='$y[n]=\sum^{\infty}_{k=-\infty}x[k]h[n-k]$')
     plt.legend()
     plt.show()
     return
xm, ym, hm = convgraf(h,x)

#@interact(m=range(xm.shape[0]-len(x)+1)) #define panel tipo selección en el gráfico interactivo
@interact(k=(0, xm.shape[0]-len(x), 1))
def show_frame(k=0):
    plot_conv(k,xm,ym,hm)
plt.show()



y_ = np.convolve(x.reshape(-1),h.reshape(-1))
y_

<>:29: SyntaxWarning: invalid escape sequence '\s'
<>:29: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-3011831673.py:29: SyntaxWarning: invalid escape sequence '\s'
  plt.stem(ym[:k,:].sum(axis=0),markerfmt='s',linefmt='r',label='$y[n]=\sum^{\infty}_{k=-\infty}x[k]h[n-k]$')
/tmp/ipython-input-3011831673.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ym[i,i] = xm[:,i].T.dot(hm) #multiplicacion entre vectores -> producto punto


interactive(children=(IntSlider(value=0, description='k', max=14), Output()), _dom_classes=('widget-interact',…

array([-105,  275, -296,  143,  151, -260,   -6,   76,  -91,  -15,  110,
          9,   -3])

**Ejercicio 2:** Repita el proceso para el sistema con respuesta al escalón $ \{-1, 6, -10, 3^\dagger, 1, -10, 2, 5\} $

In [ ]:
#crear señales
#h = np.array([1,-2,0,1,-2]).reshape(-1,1)#np.r_[np.zeros((5,1)),np.ones((5,1)),np.zeros((5,1))] # se genera un pulso rectangular
hu = np.array([-1, 6, -10, 3, 1, -10, 2, 5])
s_padded = np.r_[0, hu, 0]
h = np.diff(s_padded).reshape(-1, 1)
x = np.array([-15, 5, -3, 0, 5, 7, -1]).reshape(-1,1)#signal.triang(12).reshape(-1,1) # se genera una señal triangular y se trabaja con arreglos tipo columna
#funcion convolucion
def convgraf(h,x): #función general para pintar proceso de convolución de forma interactiva
   lx = len(x)
   lh = len(h)
   M = 2*lx+lh # se genera un vector de tamaño completo para visualizar todo el proceso paso a paso
   xm = np.zeros((M,M))
   hm = np.r_[np.zeros((lx,1)),h,np.zeros((lx,1))]
   ym = np.zeros((M,M))
   for i in range(M-lx+1):
     #print(i)
     xm[i:i+lx,i] = np.flip(x).reshape(-1)#filp realizar el proceso de reflejo sobre arreglos discretos
     ym[i,i] = xm[:,i].T.dot(hm) #multiplicacion entre vectores -> producto punto
   return xm, ym, hm

def plot_conv(k,xm,ym,hm): #dibujar arreglos
     plt.stem(xm[:,k],markerfmt='+',label='$x[k]$')
     plt.stem(hm,linefmt='g',markerfmt='.',label='$h[n-k]$')
     plt.stem(ym[:k,:].sum(axis=0),markerfmt='s',linefmt='r',label='$y[n]=\sum^{\infty}_{k=-\infty}x[k]h[n-k]$')
     plt.legend()
     plt.show()
     return
xm, ym, hm = convgraf(h,x)
#@interact(m=range(xm.shape[0]-len(x)+1)) #define panel tipo selección en el gráfico interactivo
@interact(k=(0, xm.shape[0]-len(x), 1))
def show_frame(k=0):
    plot_conv(k,xm,ym,hm)
plt.show()

y_ = np.convolve(x.reshape(-1),h.reshape(-1))
y_

<>:24: SyntaxWarning: invalid escape sequence '\s'
<>:24: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-4087837536.py:24: SyntaxWarning: invalid escape sequence '\s'
  plt.stem(ym[:k,:].sum(axis=0),markerfmt='s',linefmt='r',label='$y[n]=\sum^{\infty}_{k=-\infty}x[k]h[n-k]$')
/tmp/ipython-input-4087837536.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ym[i,i] = xm[:,i].T.dot(hm) #multiplicacion entre vectores -> producto punto


interactive(children=(IntSlider(value=0, description='k', max=16), Output()), _dom_classes=('widget-interact',…

array([  15, -110,  278, -296,  138,  144, -259,   -6,  151, -116,    0,
        110,  -16,  -38,    5])